In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import ssl

#Importing Data that was scraped previously joined
mvpDataset=pd.read_csv('Datasets/MvpVotingUpdatedJan14.csv')

#Drop unnecessary index column
mvpDataset = mvpDataset.drop(columns=['Unnamed: 0', 'Pts Won', 'Pts Max']) 


#Include player PER, Turnovers, 3PA, FGA, eFG%
mvpDataset.columns

Index(['Player', 'Age', 'Tm', 'First', 'Share', 'G', 'MP', 'PTS', 'TRB', 'AST',
       'STL', 'BLK', 'FG%', 'FT%', 'WS', 'WS/48', 'Year', 'Team', 'W', 'L',
       'Pct'],
      dtype='object')

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context

# Define the URL template
url_template = 'https://www.basketball-reference.com/leagues/NBA_{year}_{type}.html'

# Define the types of data to retrieve
data_types = {'advanced': 1, 'per_game': 0}

# Initialize an empty list to store the dataframes
dfs = []

# Loop over the years and data types
for year in range(2021, 1979, -1):
    time.sleep(2)
    for data_type, idx in data_types.items():
        url = url_template.format(year=year, type=data_type)
        df = pd.read_html(url)[0]
        df = df.dropna(how='all')  # Drop rows with all missing values
        df = df.drop(columns=['Rk'])  # Drop the 'Rk' column
        df['Year'] = year  # Add a column for the year
        df['Type'] = data_type  # Add a column for the data type
        dfs.append(df)

# Concatenate the dataframes
mergedData = pd.concat(dfs, axis=0)

# Merge the advanced and basic stats
mergedData = mergedData.pivot(index=['Player', 'Year'], columns='Type', values=mergedData.columns[:-2])

# Reset the index
mergedData = mergedData.reset_index()



HTTPError: HTTP Error 429: Too Many Requests

In [3]:
def flatten_list(_2d_list):
    return [item for sublist in _2d_list for item in sublist] if isinstance(_2d_list, list) else [_2d_list]


def processBasketballRefUrl(url, advanced):
    n = 28 if advanced else 29
    time.sleep(2)
    soup = BeautifulSoup(urlopen(url), features="lxml")
    print(soup.title.text)
    rowsContent = soup.findAll('tr')[2:]
    columnsContent = soup.findAll('th')[:n+1]
    rowsData = [[td.getText() for td in row.findAll('td')] for row in rowsContent]
    columnsData = [column.contents for column in columnsContent[1:]]
    df = pd.DataFrame(data=rowsData, columns=flatten_list(columnsData))
    return df



def toNumeric(df):
    numericCols = ['First', 'Pts Won','Pts Max','Share','WS','WS/48','Age','G','GS','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FT%','FTA','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']
    for col in df.columns:
        if col in numericCols:
            df[col] = pd.to_numeric(df[col])
    return df